# Analysis of pcHi-C data from human CD4+ T cells and ILC3 cells

This notebook will walk through the analysis of pcHi-C data generated by Mikhail Spivakov's group. We will also analyze CD4+ alpha-beta T-cells in addition to the ILC3 data as a positive control sample.

## Imports

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from decimal import Decimal
import matplotlib.ticker as ticker
import pybedtools
import os
from os import makedirs, error

In [36]:
class ChicagoData(object):
    """Import CHiCAGO data
    """
    def __init__(self,
                 filename: str,
                 drop_off_target_bait: bool = True,
                 drop_off_target_oe: bool = True,
                 drop_trans_chrom: bool = True,
                 score_col: str = None,
                 score_val: int = 5,
                 remove_p2p: bool= True,
                 features_to_count: dict = {},
                 gene_expression: str = "",
                 nonzero_expression: bool = True,
                 dropna_expression: bool = True,
                 output_dir: str = "",
                 output_basename: str = ""
                 ):
        """Initialize the object

        Args:
            filename (str): Input CHICAGO txt file
            drop_off_target (bool, optional): Drop off target interactions. Defaults to =True.
            drop_trans_chrom (bool, optional): Drop transchromosomal interactions. Defaults to =True.
        """
        # Set filename to the input filename
        self.filename = filename
        # Set whether to drop off target baits
        self.drop_off_target_bait = drop_off_target_bait
        # Set whether to drop off target oe
        self.drop_off_target_oe = drop_off_target_oe
        # Set whether to drop transchromosomal interactions
        self.drop_trans_chrom = drop_trans_chrom
        # Score column name for filtering
        self.score_col = score_col
        # Score value threshold
        self.score_val = score_val
        # Remove promoter to promoter interactions
        self.remove_p2p = remove_p2p
        # Map feature counts to PIR if provided
        self.features_to_count = features_to_count
        # Import the gene expression matrix
        self.gene_expression = gene_expression
        # Only keep non-zero expression
        self.nonzero_expression = nonzero_expression
        # Drop na Values
        self.dropna_expression = dropna_expression
        # Set the output directory
        self.output_dir = output_dir
        # Set the output basename 
        self.basename = output_basename
        
        # Read file into DF
        self._read_file_()
        
        # Format the DF
        self._format_file_()
        
        # Filter the formatted DF
        self._filter_file_()
        
        # Get the PIR df
        self._get_PIR_df_()
        
        # Get the bait df
        self._get_bait_df_()
        
        # Get the combined df
        self._get_combined_df_()  
        
        self._get_feature_counts_()
        
        self._import_gene_counts_()
            
        self._map_feature_counts_to_genes_()
        
        self._map_ABC_counts_to_genes_()
        
        self._filter_expression_()
        
        self._get_PIR_count_v_mean_()
        
        self._write_new_chicago_data_()
                    
    def _read_file_(self):
        """Read in original file
        """
        # Read in original file and save
        self.input_df =  pd.read_csv(self.filename, sep="\t", header=0, low_memory=False)
    
    def _write_new_chicago_data_(self):
        output_dir = os.path.join(self.output_dir, "modified_chicago")
        
        output_filename = os.path.join(output_dir, f"{self.basename}_modified.tsv")
        
        self._get_dir_(output_dir)

        self.df.to_csv(output_filename, sep="\t", index=False)
        
    def _format_file_(self):
        """Format CHICAGO file
        """
        # Create a copy of the raw input to be manipulated
        df = self.input_df.copy()
        
        # Format the chromosome names
        df["baitChr"] = "chr" + df["baitChr"].apply(str)
        df["oeChr"] = "chr" + df["oeChr"].apply(str)
        
        df["oe_interval_ID"] = df["oeChr"] + ":" + \
                   df["oeStart"].apply(str) + "-" + \
                   df["oeEnd"].apply(str)

        df["bait_interval_ID"] = df["baitChr"] + ":" + \
                   df["baitStart"].apply(str) + "-" + \
                   df["baitEnd"].apply(str)

        df["interaction_ID"] = df["bait_interval_ID"] + "_" + df["oe_interval_ID"].apply(str)
        
        # Set the variable to the formatted df
        self.bait_ID = df["baitID"].unique()
        
        self.oe_ID = df["oeID"].unique()

        self.bait_interval_ID = df["bait_interval_ID"].unique()
        
        self.oe_interval_ID = df["oe_interval_ID"].unique()

        df["ABC_label"] = df["ABC.Score"].apply(lambda x: "ABC" if x > 0 else "CHiCAGO")

        self.df = df

    def _filter_file_(self):
        """Filter the formatted CHICAGO results
        """
        # Drop the off target baits
        if self.drop_off_target_bait:
            self.df[self.df["baitName"] != "off_target"]

        # Drop the off target OE names
        if self.drop_off_target_oe:
            self.df[self.df["oeName"] != "off_target"]

        # Drop the trans chromosomal interactions
        if self.drop_trans_chrom:
            self.df = self.df[self.df["dist"] != "."]
                        
            self.df = self.df[self.df["dist"].apply(float) != 0]

            self.df = self.df.dropna(subset=["dist"])

            self.df = self.df[self.df["baitChr"] == self.df["oeChr"]]

        # Filter the specific score column by a specific value
        if self.score_col:
            self.df = self.df[self.df[self.score_col] >= self.score_val]
            
        # Drop promoter to promoter interactions
        if self.remove_p2p:
            self.df = self.df[self.df.oeName == "."]
            
            self.df = self.df[~self.df.oe_interval_ID.isin(self.bait_interval_ID)]
        
    def _get_PIR_df_(self):
        """Get a DF of all PIR interactions
        """
        self.pir_df = self.df[["oeChr", "oeStart", "oeEnd", "oe_interval_ID"]].drop_duplicates(subset=["oeChr", "oeStart", "oeEnd"], keep="first")
        
        self.PIR_bt = pybedtools.BedTool.from_dataframe(self.pir_df)

    def _get_bait_df_(self):
        """Get a DF of baits
        """
        self.bait_df = self.df[["baitChr", "baitStart", "baitEnd", "bait_interval_ID"]].drop_duplicates(subset=["baitChr", "baitStart", "baitEnd"], keep="first")
        
    def _get_combined_df_(self):
        """Get a comined DF
        """
        tmp_df = self.pir_df.copy()
        tmp_df2 = self.bait_df.copy()

        tmp_df.columns = ["Chr", "Start", "Stop", "ID"]
        tmp_df2.columns = ["Chr", "Start", "Stop", "ID"]

        self.unique_features = pd.concat([tmp_df, tmp_df2])
    
    def _get_feature_counts_(self):
        """Get the counts of features that overlap PIRs and map them back to the pcHiC interaction
        """
        for file, tag in self.features_to_count.items():
            print(f"Importing {file} : Column will be saved as {tag}")
            output_intersection_dir = os.path.join(self.output_dir, "PIR_intersection")

            output_intersection_fname = os.path.join(output_intersection_dir, f"{self.basename}_PIR_intersect_{tag}.bed")
            
            # Import and convert the featuers to a bedtools
            feature_bt = pybedtools.BedTool(file)
            
            # Intersect the features to get counts
            feature_counts = self.PIR_bt.intersect(feature_bt, c=True)
            
            # Intersect the features to get overlaps (true intersections)
            feature_intersection = self.PIR_bt.intersect(feature_bt)
            
            feature_intersection_sort = feature_intersection.sort()
            
            # Convert bedtools to pandas
            feature_counts_df = feature_counts.to_dataframe()
                        
            # Convert bedtools to dataframe
            feature_intersection_df = feature_intersection_sort.to_dataframe()
                        
            # Create a dictionary of counts 
            counts_dict =  pd.Series(feature_counts_df["score"].values,index=feature_counts_df["name"]).to_dict()

            # Map the counts back to the CHICAGO dataframe
            self.df[tag] = self.df["oe_interval_ID"].map(counts_dict)
            
            self._get_dir_(output_intersection_dir)
            
            feature_intersection_df[["chrom", "start", "end"]].to_csv(output_intersection_fname, sep="\t", index=False, header=False)

    def _import_gene_counts_(self):
        self.gene_counts = pd.read_csv(self.gene_expression, sep="\t", header=0, names=["GeneName", "Expression"])
    
    def _map_feature_counts_to_genes_(self):
        self.gene_counts["enhancer_count"] = self.gene_counts["GeneName"].map(self.df.groupby(["baitName"]).count()["baitChr"])
        
        for _, tag in self.features_to_count.items():
            self.gene_counts[f"{tag}_count"] = self.gene_counts["GeneName"].map(self.df.groupby(["baitName"]).sum()[tag])

    def _map_ABC_counts_to_genes_(self):        
        for _, tag in self.features_to_count.items():
            tmp = self.df[self.df["ABC_label"] == "ABC"]
            
            self.gene_counts[f"{tag}_ABC_count"] = self.gene_counts["GeneName"].map(tmp.groupby(["baitName"]).sum()[tag])

            self.gene_counts[f"{tag}_CHiCAGO_count"] = self.gene_counts[f"{tag}_count"] - self.gene_counts[f"{tag}_ABC_count"]

    def _get_dir_(self, dir: str, permissions=0o0775, exist_ok : bool=True):
        """Makes a directory at the given location
        Args:
            dir (str): Path of the directory
            permissions ([type], optional): Permissions of directory. Defaults to 0o0775.
            exist_ok (bool, optional): If True, program will continue if directory exists. Defaults to True.
        Returns:
            str: Absolute path to the created directory
            
        Example:
        
        >>> output_dir = get_dir("./output/")
        """
        try:
            makedirs(dir, mode=permissions)
        except error:
            if not exist_ok:
                raise

    def _filter_expression_(self):
        output_analysis_dir = os.path.join(self.output_dir, "expression_matrix")

        output_analysis_fname = os.path.join(output_analysis_dir, f"{self.basename}_expression_matrix.tsv")
           
        if self.nonzero_expression:
            self.gene_counts = self.gene_counts[self.gene_counts["Expression"] > 0]

        if self.dropna_expression:
            self.gene_counts = self.gene_counts.dropna()

        self.gene_counts["GeneName_MeanExpression"] = self.gene_counts["GeneName"] \
            + " " + self.gene_counts["Expression"].apply(str)
            
        self._get_dir_(output_analysis_dir)

        self.gene_counts.to_csv(output_analysis_fname, sep="\t", index=False)
        
    def _calculate_spearman_(self):
        self.corr = []
        output_analysis_dir = os.path.join(self.output_dir, "correlation_analysis")

        output_analysis_fname = os.path.join(output_analysis_dir, f"{self.basename}_correlation_stats.tsv")
             
        for _, tag in self.features_to_count.items():
            tmp_df = self.gene_counts[[f"{tag}_count", "Expression"]].copy()
            
            s_corr, s_pval = spearmanr(tmp_df[f"{tag}_count"], tmp_df["Expression"])
            
            p_corr, p_pval = pearsonr(tmp_df[f"{tag}_count"], tmp_df["Expression"])

            self.corr.append([s_corr, s_pval, p_corr, p_pval, tag])

        self.corr_df = pd.DataFrame(self.corr, columns=["Spearman_corr", "Spearman_pval", "Pearson_corr", "Pearson_pval", "Feature"])
            
        self._get_dir_(output_analysis_dir)

        self.corr_df.to_csv(output_analysis_fname, sep="\t", index=False)

    def _get_PIR_count_v_mean_(self):
        """Create the dataframe of the number of features overlapping PIRs by mean gene expression
        
        First you groupby the feature counts column. Then you find all of the mean gene expression values
        associated with the number of features overlapping a PIR. The output is a dataframe that can be plotted
        """
    
        for _, tag in self.features_to_count.items():
            col_name = f"{tag}_count"
            output_analysis_dir = os.path.join(self.output_dir, "expression_analysis")

            output_analysis_fname = os.path.join(output_analysis_dir, f"{self.basename}_{tag}.tsv")
            
            # Create the dataframe
            self.pir_count_v_mean = pd.DataFrame(self.gene_counts.groupby([col_name])["GeneName_MeanExpression"].apply(list)).reset_index().explode("GeneName_MeanExpression")
        
            self.pir_count_v_mean[col_name] = self.pir_count_v_mean[col_name].apply(int)
            
            self.pir_count_v_mean[["Gene_Name", "Mean_Gene_Expression"]] = self.pir_count_v_mean["GeneName_MeanExpression"].str.split(" ", n=2, expand=True)
            
            self.pir_count_v_mean.drop("GeneName_MeanExpression",axis=1, inplace=True)
            
            self.pir_count_v_mean["Mean_Gene_Expression"] = self.pir_count_v_mean["Mean_Gene_Expression"].apply(float)
            
            self._get_dir_(output_analysis_dir)

            self._calculate_spearman_()
            
            self.pir_count_v_mean.to_csv(output_analysis_fname, sep="\t", index=False)


## Getting PIRs From CHiCAGO output file

I wrote a python class object to work with the CHiCAGO output text file. This piece of code will perform filtering of specific types of interactions, like promoter-to-promoter, or trans-chromosomal interactions. 

Example:

```python
input_file = "../data/CHICAGO/hg38/inputs/ILC_5kb_within_newbmap_CHiCAGO_ABC_peakm.txt"

ILC3_data = ChicagoData(input_file)

ILC3_data = ChicagoData(input_file, 
                        drop_off_target_bait=True, 
                        drop_off_target_oe=False, 
                        drop_trans_chrom=True,
                        score_col="merged_score",
                        score_val=5,
                        remove_p2p=True)
                        
ILC3_data.pir_df
```

In [29]:
# Input Chicago File
input_file = "../data/CHICAGO/hg38/inputs/ILC_5kb_within_newbmap_CHiCAGO_ABC_peakm.txt"

In [30]:
ILC3_data = ChicagoData(input_file, 
                        drop_off_target_bait=True, 
                        drop_off_target_oe=False, 
                        drop_trans_chrom=True,
                        score_col="merged_score",
                        score_val=5,
                        remove_p2p=True,
                        features_to_count=ilc3_file_dict,
                        gene_expression="/Users/caz3so/workspaces/tacazares/pchic/data/RNA/ILC3_mean_expression.tsv",
                        output_dir="/Users/caz3so/workspaces/tacazares/pchic/data/outputs",
                        output_basename="ILC_5kb_within_newbmap_CHiCAGO_ABC_peakm")

Importing /Users/caz3so/workspaces/tacazares/pchic/data/peaks/ATAC/ILC3_ATAC_peaks.bed : Column will be saved as ATAC
Importing /Users/caz3so/workspaces/tacazares/pchic/data/peaks/CHIP/ILC3_H3K27ac_peaks.bed : Column will be saved as H3K27ac
Importing /Users/caz3so/workspaces/tacazares/pchic/data/peaks/CHIP/ILC3_H3K4me3_peaks.bed : Column will be saved as H3K4me3
Importing /Users/caz3so/workspaces/tacazares/pchic/data/peaks/RE/ILC3_RE.bed : Column will be saved as RE


In [37]:
ILC3_data.pir_count_v_mean

,RE_count,Gene_Name,Mean_Gene_Expression
0,0,A1CF,5.166667
0,0,A2MP1,4.833333
0,0,AADAT,3.666667
0,0,AARD,81.833333
0,0,ABCA13,57.166667
...,...,...,...
97,148,TTF2,1940.666667
98,152,CD44,12834.500000
98,152,NSMCE1,8499.833333
99,320,ID2,24945.500000


## Processing Files

In [8]:
pwd

'/Users/caz3so/workspaces/tacazares/pchic/notebooks'

In [31]:
ilc3_file = "../data/CHICAGO/hg38/inputs/ILC_5kb_within_newbmap_CHiCAGO_ABC_peakm.txt"
ilc3_old_file = "../data/CHICAGO/hg38/inputs/ILC3_merged_bin5K_score5.txt"

cd4_1M_old_file = "../data/CHICAGO/hg38/inputs/CD4_1M_50K_merged_reweighting_peakmatrix_score5.txt"
cd4_file = "../data/CHICAGO/hg38/inputs/CD4_1M_50K_5kb_within_newbmap_CHiCAGO_peakm.txt"
cd4_ABC = "../data/CHICAGO/hg38/inputs/CD4_1M_50K_5kb_within_newbmap_CHiCAGO_ABC_peakm.txt"

In [32]:
ilc3_file_dict = {"/Users/caz3so/workspaces/tacazares/pchic/data/peaks/ATAC/ILC3_ATAC_peaks.bed": "ATAC",
                  "/Users/caz3so/workspaces/tacazares/pchic/data/peaks/CHIP/ILC3_H3K27ac_peaks.bed": "H3K27ac",
                  "/Users/caz3so/workspaces/tacazares/pchic/data/peaks/CHIP/ILC3_H3K4me3_peaks.bed": "H3K4me3",
                  "/Users/caz3so/workspaces/tacazares/pchic/data/peaks/RE/ILC3_RE.bed": "RE"}

cd4_file_dict = {"/Users/caz3so/workspaces/tacazares/pchic/data/peaks/ATAC/CD4_ATAC_peaks.bed": "ATAC",
                 "/Users/caz3so/workspaces/tacazares/pchic/data/peaks/CHIP/S008H1H1.ERX547940.H3K27ac.bwa.GRCh38.20150527.bed": "H3K27ac",
                 "/Users/caz3so/workspaces/tacazares/pchic/data/peaks/CHIP/S008H1H1.ERX547958.H3K4me3.bwa.GRCh38.20150527.bed": "H3K4me3",
                 "/Users/caz3so/workspaces/tacazares/pchic/data/peaks/RE/CD4_RE.bed": "RE"}

In [33]:
ilc_gene_expression = "/Users/caz3so/workspaces/tacazares/pchic/data/RNA/ILC3_mean_expression.tsv"
cd4_gene_expression = "/Users/caz3so/workspaces/tacazares/pchic/data/RNA/CD4_mean_expression.tsv"

In [34]:
out_dir = "../data/outputs"

In [35]:
# Import ILC3 data
ilc3 = ChicagoData(ilc3_file, 
                        drop_off_target_bait=True, 
                        drop_off_target_oe=False, 
                        drop_trans_chrom=True,
                        score_col="merged_score",
                        score_val=5,
                        remove_p2p=True,
                        features_to_count=ilc3_file_dict,
                        gene_expression=ilc_gene_expression,
                        output_dir=out_dir,
                        output_basename="ILC_5kb_within_newbmap_CHiCAGO_ABC_peakm")

# Import cd4 file CD4_1M_50K_5kb_within_newbmap_CHiCAGO_peakm.txt
cd4 = ChicagoData(cd4_file, 
                        drop_off_target_bait=True, 
                        drop_off_target_oe=False, 
                        drop_trans_chrom=True,
                        score_col="merged_score",
                        score_val=5,
                        remove_p2p=True,
                        features_to_count=cd4_file_dict,
                        gene_expression=cd4_gene_expression,
                        output_dir=out_dir,
                        output_basename="CD4_1M_50K_5kb_within_newbmap_CHiCAGO_peakm")

# Import cd4_abc file CD4_1M_50K_5kb_within_newbmap_CHiCAGO_ABC_peakm.txt
cd4_abc = ChicagoData(cd4_ABC, 
                        drop_off_target_bait=True, 
                        drop_off_target_oe=False, 
                        drop_trans_chrom=True,
                        score_col="merged_score",
                        score_val=5,
                        remove_p2p=True,
                        features_to_count=cd4_file_dict,
                        gene_expression=cd4_gene_expression,
                        output_dir=out_dir,
                        output_basename="CD4_1M_50K_5kb_within_newbmap_CHiCAGO_ABC_peakm")

# Import old ILC3 data
ilc3_old = ChicagoData(ilc3_old_file, 
                        drop_off_target_bait=True, 
                        drop_off_target_oe=False, 
                        drop_trans_chrom=True,
                        score_col="hILC3_merged_bin5k",
                        score_val=5,
                        remove_p2p=True,
                        features_to_count=ilc3_file_dict,
                        gene_expression=ilc_gene_expression,
                        output_dir=out_dir,
                        output_basename="ILC3_merged_bin5K_score5")

# Import old CD4_1M data
cd4_1M_old = ChicagoData(cd4_1M_old_file, 
                        drop_off_target_bait=True, 
                        drop_off_target_oe=False, 
                        drop_trans_chrom=True,
                        score_col="M1",
                        score_val=5,
                        remove_p2p=True,
                        features_to_count=cd4_file_dict,
                        gene_expression=cd4_gene_expression,
                        output_dir=out_dir,
                        output_basename="CD4_1M_50K_merged_reweighting_peakmatrix_score5")


Importing /Users/caz3so/workspaces/tacazares/pchic/data/peaks/ATAC/ILC3_ATAC_peaks.bed : Column will be saved as ATAC
Importing /Users/caz3so/workspaces/tacazares/pchic/data/peaks/CHIP/ILC3_H3K27ac_peaks.bed : Column will be saved as H3K27ac
Importing /Users/caz3so/workspaces/tacazares/pchic/data/peaks/CHIP/ILC3_H3K4me3_peaks.bed : Column will be saved as H3K4me3
Importing /Users/caz3so/workspaces/tacazares/pchic/data/peaks/RE/ILC3_RE.bed : Column will be saved as RE
Importing /Users/caz3so/workspaces/tacazares/pchic/data/peaks/ATAC/CD4_ATAC_peaks.bed : Column will be saved as ATAC
Importing /Users/caz3so/workspaces/tacazares/pchic/data/peaks/CHIP/S008H1H1.ERX547940.H3K27ac.bwa.GRCh38.20150527.bed : Column will be saved as H3K27ac
Importing /Users/caz3so/workspaces/tacazares/pchic/data/peaks/CHIP/S008H1H1.ERX547958.H3K4me3.bwa.GRCh38.20150527.bed : Column will be saved as H3K4me3
Importing /Users/caz3so/workspaces/tacazares/pchic/data/peaks/RE/CD4_RE.bed : Column will be saved as RE
Im

KeyError: 'ABC.Score'

In [178]:
cd4_1M_old.corr_df

,Spearman_corr,Spearman_pval,Pearson_corr,Pearson_pval,Feature
0,0.227638,2.493171e-67,0.039472,0.003000,ATAC
1,0.114975,4.326785e-18,0.024818,0.062110,H3K27ac
2,0.167423,8.403177e-37,0.028969,0.029429,H3K4me3
3,0.229787,1.318676e-68,0.039183,0.003219,RE
